In [1]:
#need to sudo apt-get install tesseract-ocr
#need to also install poppler-utils
import encodings
from PIL import Image 
#import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 
import pandas as pd, math, inspect, requests
from lxml import html
import re, json, io
import textract
import urllib.parse
import scrapy
from scrapy.crawler import CrawlerProcess
from PyPDF2 import PdfReader
import os
import urllib
from urllib.error import HTTPError
import ssl
import pdfpages
import pikepdf
#from docx2pdf import convert
from datetime import date
today = date.today().strftime("%Y-%m-%d")
import pdfplumber
from pdfminer.pdfdocument import PDFDocument
from pytesseract import image_to_string

In [2]:
#Document List scraper. 
#Run after and independently of document folder scraper

#read existing document data
dfcsv = pd.read_csv('documents.csv')
dfcsv['doctype']=dfcsv['url'].str.split('.').str[-1].str.lower()
dfcsv = dfcsv[dfcsv['filename'].str.contains('00132553')==False]

dfcsv

,url,filename,pdc_document_type,document_id,project_id,id,document_category,operating_unit,doctype,eval1,...,end_date_cond,budget_cond,concept_note_cond,project_document_cond,awp_cond,title,modified,created,last_update,UN Languages
0,https://info.undp.org/docs/pdc/Documents/ARE/0...,00094357 PRODOC Gender Mainstreaming,Prodoc,ATLASPDC-4-59365,00094357,1191009,Project,ARE,pdf,good,...,True,True,False,False,False,00094357 PRODOC Gender Mainstreaming,1/15/2017 8:17 AM,1/15/2017 8:13 AM,28/09/20,NaN
1,https://info.undp.org/docs/pdc/Documents/ABW/S...,Signed prodoc -Centre of Excellence for SIDS i...,Prodoc,ATLASPDC-4-70038,00091538,1746061,Project,ABW,pdf,good,...,True,True,False,True,False,Aruba Centre of Excellence Signed Prodoc,10/09/17 17:38,10/09/17 17:24,10/03/21,NaN
2,https://info.undp.org/docs/pdc/Documents/ARE/A...,Addendum no lll_Project Document,Prodoc,ATLASPDC-4-19429,00061885,1746438,Project,ARE,pdf,bad,...,False,True,False,True,True,NaN,6/29/2014 1:37 AM,6/29/2014 1:37 AM,10/03/21,NaN
3,https://info.undp.org/docs/pdc/Documents/ARE/A...,AD-HDR 2010-2011 ProDoc Signed,Prodoc,ATLASPDC-4-19430,00060650,1746441,Project,ARE,pdf,good,...,True,True,True,False,True,NaN,6/29/2014 1:51 AM,6/29/2014 1:49 AM,10/03/21,NaN
4,https://info.undp.org/docs/pdc/Documents/ARE/A...,AD-HDR 2010-2011 ProDoc Signed-Arabic,Prodoc,ATLASPDC-4-19431,00060650,1746442,Project,ARE,pdf,bad,...,False,False,False,False,False,NaN,6/29/2014 2:07 AM,6/29/2014 1:53 AM,10/03/21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19146,https://undpngddlsprod01.blob.core.windows.net...,Prodoc ESI revision 1 - final ENG.pdf,Prodoc,168091,00113134,168091,Project,SRB,pdf,good,...,True,True,True,True,True,NaN,28/12/22,28/12/22,19/02/24,English
19147,https://undpngddlsprod01.blob.core.windows.net...,Annex 1-DoA IPA 2021_Health FINAL CLEAN.docx,Prodoc,168431,00136378,168431,Project,SRB,docx,bad,...,False,True,False,False,True,NaN,06/01/23,06/01/23,19/02/24,English
19148,https://undpngddlsprod01.blob.core.windows.net...,2022_1_ProDoc_UNDP MKD.docx.pdf,Prodoc,168498,00143865,168498,Project,MKD,pdf,good,...,True,True,True,True,True,ProDoc,16/01/23,16/01/23,19/02/24,English
19149,https://undpngddlsprod01.blob.core.windows.net...,00130257_PRODOC.pdf,Prodoc,166433,00141763,166433,Project,MEX,pdf,good,...,True,True,True,True,False,PRODOC,15/11/22,15/11/22,19/02/24,English


In [4]:

dfcsv_newdocs = pd.read_csv('temp-documents-riza.csv',lineterminator='\n' )
# dfcsv_newdocs = pd.read_csv('temp-documents-riza.csv', lineterminator='\n', on_bad_lines='skip' )
dfcsv_newdocs.columns = dfcsv_newdocs.columns.astype(str)
dfcsv_newdocs['doctype']= dfcsv_newdocs['url'].str.split('.').str[-1].str.lower()
dfcsv_newdocs['last_update'] = today

#Remove extra columns

dfcsv_newdocs = dfcsv_newdocs.drop(['Sharepoint URL','Classification Level', 'Modified By', 'Document Status', 'Checked Out To', 'Content Type\r'], axis=1)

dfcsv_newdocs


,doctype,filename,url,title,operating_unit,project_id,document_category,pdc_document_type,document_id,modified,created,UN Languages,id,last_update
0,pdf,Project Document ESPECRP IDA 19 FINAL.doc.pdf,https://undpngddlsprod01.blob.core.windows.net...,ESPCRP - AF,YEM,00141815,Project,Prodoc,165076,26/09/22,26/09/22,English,165076,2024-02-19
1,pdf,Initiation Plan GCP Phase II.pdf,https://undpngddlsprod01.blob.core.windows.net...,Initiation Plan GCP Phase II,R46,00123562,Project,Prodoc,146602,22/02/22,22/02/22,English,146602,2024-02-19
2,pdf,BRA14G31 - 2021.05.28 - General Revision 13 (s...,https://undpngddlsprod01.blob.core.windows.net...,BRA14G31 - 2021.05.28 - General Revision 13 (s...,BRA,00077784,Project,Prodoc,163348,21/07/22,21/07/22,English,163348,2024-02-19
3,pdf,AGR ILO UNPRPD - Output 00130305.pdf,https://undpngddlsprod01.blob.core.windows.net...,Agreement signed + Term of Reference - Output ...,BRA,00121537,Project,Prodoc,148721,08/03/22,08/03/22,English,148721,2024-02-19
4,pdf,BRA14G31 - Revision 2022.06.30 - General 14 si...,https://undpngddlsprod01.blob.core.windows.net...,BRA14G31 - Revision 2022.06.30 - General 14 si...,BRA,00077784,Project,Prodoc,163349,21/07/22,21/07/22,English,163349,2024-02-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,pdf,ProDoc__CEB_TA for STP Banja Luka UNDP Format.pdf,https://undpngddlsprod01.blob.core.windows.net...,Pro DoC,BIH,00143020,Project,Prodoc,166856,17/12/22,30/11/22,English,166856,2024-02-19
322,pdf,124302_Revisión Final E.pdf,https://undpngddlsprod01.blob.core.windows.net...,ARG20010 - Revisión Final E,ARG,00124302,Project,Prodoc,166908,01/12/22,01/12/22,Spanish,166908,2024-02-19
323,pdf,SNHRC-project Document.pdf,https://undpngddlsprod01.blob.core.windows.net...,SNHRC-project Document,SDN,00087114,Project,Prodoc,165466,13/10/22,13/10/22,English,165466,2024-02-19
324,pdf,1 103437_PIMS 6275 ProDoc SIGNED_ABS.pdf,https://undpngddlsprod01.blob.core.windows.net...,Signed ProDoc,PHL,00100511,Project,Prodoc,169250,26/04/23,26/04/23,English,169250,2024-02-19


In [5]:


new_docs = dfcsv_newdocs[~dfcsv_newdocs['document_id'].isin(dfcsv['document_id'])]

# dfcsv = pd.concat([new_docs , dfcsv], ignore_index=True)
# dfcsv = dfcsv._append(dfcsv_newdocs[dfcsv_newdocs['document_id'].isin(dfcsv['document_id'])==False],ignore_index=True)
 dfcsv = dfcsv._append(new_docs,ignore_index=True)

In [6]:
dfcsv.iloc[:, :] 

,url,filename,pdc_document_type,document_id,project_id,id,document_category,operating_unit,doctype,eval1,...,end_date_cond,budget_cond,concept_note_cond,project_document_cond,awp_cond,title,modified,created,last_update,UN Languages
0,https://info.undp.org/docs/pdc/Documents/ARE/0...,00094357 PRODOC Gender Mainstreaming,Prodoc,ATLASPDC-4-59365,00094357,1191009,Project,ARE,pdf,good,...,True,True,False,False,False,00094357 PRODOC Gender Mainstreaming,1/15/2017 8:17 AM,1/15/2017 8:13 AM,28/09/20,NaN
1,https://info.undp.org/docs/pdc/Documents/ABW/S...,Signed prodoc -Centre of Excellence for SIDS i...,Prodoc,ATLASPDC-4-70038,00091538,1746061,Project,ABW,pdf,good,...,True,True,False,True,False,Aruba Centre of Excellence Signed Prodoc,10/09/17 17:38,10/09/17 17:24,10/03/21,NaN
2,https://info.undp.org/docs/pdc/Documents/ARE/A...,Addendum no lll_Project Document,Prodoc,ATLASPDC-4-19429,00061885,1746438,Project,ARE,pdf,bad,...,False,True,False,True,True,NaN,6/29/2014 1:37 AM,6/29/2014 1:37 AM,10/03/21,NaN
3,https://info.undp.org/docs/pdc/Documents/ARE/A...,AD-HDR 2010-2011 ProDoc Signed,Prodoc,ATLASPDC-4-19430,00060650,1746441,Project,ARE,pdf,good,...,True,True,True,False,True,NaN,6/29/2014 1:51 AM,6/29/2014 1:49 AM,10/03/21,NaN
4,https://info.undp.org/docs/pdc/Documents/ARE/A...,AD-HDR 2010-2011 ProDoc Signed-Arabic,Prodoc,ATLASPDC-4-19431,00060650,1746442,Project,ARE,pdf,bad,...,False,False,False,False,False,NaN,6/29/2014 2:07 AM,6/29/2014 1:53 AM,10/03/21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19472,https://undpngddlsprod01.blob.core.windows.net...,ProDoc__CEB_TA for STP Banja Luka UNDP Format.pdf,Prodoc,166856,00143020,166856,Project,BIH,pdf,NaN,...,NaN,NaN,NaN,NaN,NaN,Pro DoC,17/12/22,30/11/22,2024-02-19,English
19473,https://undpngddlsprod01.blob.core.windows.net...,124302_Revisión Final E.pdf,Prodoc,166908,00124302,166908,Project,ARG,pdf,NaN,...,NaN,NaN,NaN,NaN,NaN,ARG20010 - Revisión Final E,01/12/22,01/12/22,2024-02-19,Spanish
19474,https://undpngddlsprod01.blob.core.windows.net...,SNHRC-project Document.pdf,Prodoc,165466,00087114,165466,Project,SDN,pdf,NaN,...,NaN,NaN,NaN,NaN,NaN,SNHRC-project Document,13/10/22,13/10/22,2024-02-19,English
19475,https://undpngddlsprod01.blob.core.windows.net...,1 103437_PIMS 6275 ProDoc SIGNED_ABS.pdf,Prodoc,169250,00100511,169250,Project,PHL,pdf,NaN,...,NaN,NaN,NaN,NaN,NaN,Signed ProDoc,26/04/23,26/04/23,2024-02-19,English


In [7]:
project_title = ['project title','title of the program','programme title', 'title of program', 'award title', 'de projet', 'titulo proyecto', 'titulo de proyecto', 'título proyecto', 'de proyecto','titulo del proyecto', 'título del proyecto', 'nombre del proyecto','title of action', 'title of the action', 'título del programa', 'titulo del programa']
summary_desc = ['summary', 'project description', 'brief description','breve description', 'breve descripcion', 'breve descripción', 'resume de projet','resume', 'résumé', 'resumen', 'descripción del programa', 'descripcion del programa', 'description of the action']
project_id = ['award id', 'project no','project number','numero de projet','identifiant','du projet','numero proyecto','projecto id','número del proyecto','numero del proyecto', 'project id']
implementing_partner = ['implementing partner', 'executing partner', 'implementing organization','implementing un organization', 'implementing organisation', 'implementing agenc', 'executing agenc', 'partenaire dexecution', 'partenaire d’exécution' ,'agence dexecution', 'agence d’exécution', 'agencia ejecutora', 'asociado en la implementación', 'socio del programa', 'socio(s) del programa','socios de implementacion', 'socios de implementación']
start_date = ['commencing date','start date', 'commencement date', 'starting date','start year','duration of program', 'period,duration', 'date de debut', 'date de début','periode', 'periodo','fecha de inicio', 'ano inicio', 'inicio del proyecto' ]
end_date = ['ending date','end date', 'period end date', 'end year', 'project period','duration of program','project duration', 'date de fin', 'fin de periode', 'fin de période', 'fecha final', 'ano finalizacion','fecha de finalización','fecha de finalizacion', 'fecha de termino']
budget = ['total budget','total resources', 'total fund', 'required budget', 'estimated resources', 'resources required', 'total','financial','budget requis', 'total financiamiento', 'total financiero','presupuesto']

#additional metrics to determine if document is "OK"
#If the document quality is good but the total page count is less than or equal to 4 pages, it will be “Not Ok”
#If the document quality is good and the total page count is more than 4, it is “Ok”

#if document good but not ok (<5 pages) if conecept note passed then change to OK.
concept_note = ['background','contexte','fondo','rationale','raisonnement','concept','concepto','situation analysis','context analysis', 'documento de proyecto','document de projet','analyse de la situation','análisis de situación', 'initiation plan','plan de iniciación',"plan d'initiation", 'análisis del contexto', 'analisis del contexto']

#if document >5 pages but not good: if project_document passes then document passes to OK & good.
project_document = ['project document', 'documento de proyecto', 'document de projet', 'documento del programa', 'documento del proyecto']

awp= ['annual workplan', 'annual work plan', 'plan de trabajo','annual plan','de travail annuel','combined delivery report', 'rapport de livraison combiné','informe de entrega combinado','cdr','awp', 'resultados esperados', 'action plan']

#if bad  & >10 pages then review
# if bad  & >5 pages and (concept note or project note TRUE) then OK
#if bad & 5<pages<=10 then review
#if bad & 5<pages<=10 & awp = True  then bad

In [8]:
dic1 = {'√©': 'é',
'√°': 'á',
'√≠': 'í',
'√≥': 'ó',
'√∂': 'ö',
'√º': 'ü',
'√¥': 'ô',
'√®': 'è',
'√ß': 'ç',
'√±': 'ñ',
'√∏': 'ø',
'√´': 'ë',
'√§': 'ä',
'√•': 'å',
'√Å': 'Á',
'√∫': 'ú',
'√ª': 'û',
'√Ø': 'ï',
'√â': 'É',
'√†': 'à',
'√¶': 'æ',
'√Æ': 'î',
'√¢': 'â',
'√£': 'ã',
'√î': 'Ô',
'√ü': 'ß',
'√ì': 'Ó',
'√≤': 'ò',
'√Ω': 'ý',
'√ñ': 'Ö',
'√™': 'ê',
'√Ä': 'À',
'√ò': 'Ø',
'√Ö': 'Å',
'√∞': 'ð',
'√á': 'Ç',
'√Ç': 'Â',
'√π': 'ù',
'√í': 'Ò',
'√¨': 'ì',
'√ú': 'Ü',
'√à': 'È',
'√û': 'Þ',
'√ö':'Ú',
'°':'¬∞'}

In [9]:
for old, new in dic1.items():
    dfcsv['url'] = dfcsv['url'].str.replace(old, new, regex=False)

In [10]:
len(dfcsv[dfcsv['eval1'].isnull() & dfcsv['eval2'].isnull()])

326

In [11]:
def convert_pdf_to_img(pdf_file):
    """
    @desc: this function converts a PDF into Image
    
    @params:
        - pdf_file: the file to be converted
    
    @returns:
        - an interable containing image format of all the pages of the PDF
    """
    return convert_from_path(pdf_file, first_page=1, last_page=10)


def convert_image_to_text(file):
    """
    @desc: this function extracts text from image
    
    @params:
        - file: the image file to extract the content
    
    @returns:
        - the textual content of single image
    """
    
    text = image_to_string(file)
    return text


def get_text_from_any_pdf(pdf_file):
    """
    @desc: this function is our final system combining the previous functions
    
    @params:
        - file: the original PDF File
    
    @returns:
        - the textual content of ALL the pages
    """
    images = convert_pdf_to_img(pdf_file)
    final_text = ""
    for pg, img in enumerate(images):
        
        final_text += convert_image_to_text(img)
        #print("Page n°{}".format(pg))
        #print(convert_image_to_text(img))

    final_text = final_text.lower()
    return final_text

# print(get_text_from_any_pdf(url2))

In [12]:
i=0
for index,docrow in dfcsv[dfcsv['eval1'].isnull() & dfcsv['eval2'].isnull()].head(400).iterrows():
    i=i+1
    context = ssl._create_unverified_context()
    print(docrow['url'])
    #download file - to do so need to replace spaces with defined spaces
    url = docrow['url']
    url = urllib.parse.urlsplit(url)
    url = list(url)
    url[2] = urllib.parse.quote(url[2])
    url = urllib.parse.urlunsplit(url)
    page_content=""
    passer2 =""
    content_missing = ""
    
    #reading file, depending on filetype
    try:
            downloadfile = urllib.request.urlopen(url,context=context)
        
            if docrow['doctype'] =='pdf':
                file = open("temp-file.pdf", 'wb')
                file.write(downloadfile.read())
                try:
                    pdfReader = PdfReader("temp-file.pdf")
                    passer = True
                except Exception as error:
                    print("An error occurred:", error)
        
                    print("A: File Corrupt: ", url)
                    eval2 ='corrupt file'
                    eval1 = 'bad'
                    passer = False
                file.close()
                    
                if passer == True:
                        
                     #   try:
                            #counting pages
                        pages = len(pdfReader.pages)
        
                            #get content from first 5 pages
                       # for page_number in range(0,min(5,pages)):
                       # page = pdfReader.pages[page_number]
                        page_content = get_text_from_any_pdf("temp-file.pdf")
                    #    except:
                    #        print("no hay contenido")
        
                    #       try:
                                #sometimes it works the second time around
                     #           pages = pdfReader.numPages
                     #       except:
                     #           try:
                     #               with pdfplumber.open("temp-file.pdf") as pdf:
                     #                   pages = len(pdf.pages)
                     #           except:
                     #               pdf = pikepdf.Pdf.open('temp-file.pdf')
                     #               pages = len(pdf.pages)
                                
                        try: #check if it is not ok or review, if that's the case runn through pike or plumber.
                            project_title_cond  = bool([ele for ele in project_title if(ele in page_content)])
                            summary_desc_cond  = bool([ele for ele in summary_desc if(ele in page_content)])
                            project_id_cond  = bool([ele for ele in project_id if(ele in page_content)])
                            implementing_partner_cond  = bool([ele for ele in implementing_partner if(ele in page_content)])
                            start_date_cond  = bool([ele for ele in start_date if(ele in page_content)])
                            end_date_cond  = bool([ele for ele in end_date if(ele in page_content)])
                            budget_cond  = bool([ele for ele in budget if(ele in page_content)])
                            project_document_cond = bool([ele for ele in project_document if(ele in page_content)])
                            concept_note_cond = bool([ele for ele in concept_note if(ele in page_content)])
                            awp_cond = bool([ele for ele in awp if(ele in page_content)])
            
                            score = project_title_cond + summary_desc_cond +project_id_cond +implementing_partner_cond +start_date_cond +end_date_cond + budget_cond
                        except Exception as error:
                            print("An error occurred:", error)
        
                            score = 0
                                    
                                                  
                os.remove("temp-file.pdf")
        
        
            elif docrow['doctype'] == 'docx':
                file = open("temp-file.docx", 'wb')
                file.write(downloadfile.read())
        
                text = textract.process("temp-file.docx")
                file.close()
                os.remove("temp-file.docx")
                
                #estimate number of pages based on character count. This is not accurate, but approximate
                pages = round(len(text)/2000)
                
                #get content from first 5 pages (approx)
                page_content = str(text[0:12000]).lower()
                
            elif docrow['doctype'] == 'doc':
                try:
                    try:
                        file = open("temp-file.doc", 'wb')
                        file.write(downloadfile.read())
        
                        text = textract.process("temp-file.doc")
                        file.close()
                        os.remove("temp-file.doc")
        
                        #estimate number of pages based on character count. This is not accurate, but approximate
                        pages = round(len(text)/2000)
        
                        #get content from first 5 pages (approx)
                        page_content = str(text[0:12000]).lower()
                    except:
                        try:
                            file.close()
                        except:
                            pass
                        try:
                            os.remove("temp-file.doc")
                        except:
                            pass
                        file = open("temp-file.docx", 'wb')
                        file.write(downloadfile.read())
        
                        text = textract.process("temp-file.docx")
                        file.close()
                        os.remove("temp-file.docx")
        
                        #estimate number of pages based on character count. This is not accurate, but approximate
                        pages = round(len(text)/2000)
        
                        #get content from first 5 pages (approx)
                        page_content = str(text[0:12000]).lower()
                except Exception as error:
                    print("An error occurred:", error)
                    print("C: File corrupt: ", url)
                
            else:
                page_content = ""
                pages = 0
            
    except Exception as error:
            print("An error occurred:", error)
            print("X: File corrupt: ", url)
            eval2 ='corrupt file'
            eval1 = 'bad'
            passer = False



    #Scoring mechanism based on keyword searches    
    project_title_cond  = bool([ele for ele in project_title if(ele in page_content)])
    summary_desc_cond  = bool([ele for ele in summary_desc if(ele in page_content)])
    project_id_cond  = bool([ele for ele in project_id if(ele in page_content)])
    implementing_partner_cond  = bool([ele for ele in implementing_partner if(ele in page_content)])
    start_date_cond  = bool([ele for ele in start_date if(ele in page_content)])
    end_date_cond  = bool([ele for ele in end_date if(ele in page_content)])
    budget_cond  = bool([ele for ele in budget if(ele in page_content)])
    project_document_cond = bool([ele for ele in project_document if(ele in page_content)])
    concept_note_cond = bool([ele for ele in concept_note if(ele in page_content)])
    awp_cond = bool([ele for ele in awp if(ele in page_content)])

    score = project_title_cond + summary_desc_cond +project_id_cond +implementing_partner_cond +start_date_cond +end_date_cond + budget_cond

    if score>=4:
        eval1 = 'good'
    else:
        eval1 = 'bad'

    if (pages > 4) & (eval1 == 'good'):
        eval2 = 'ok'
    else:
        eval2 = 'not ok'

    if (eval1 == 'good') & (eval2 == 'not ok'):
        if concept_note_cond == True:
            eval2 = 'ok'

    if (eval1 == 'bad') & (eval2 == 'ok'):
        if project_document_cond == True:
            eval1 = 'good'
            
    if (eval1 == 'bad')  & (pages >10 ):
        eval2 = 'review'
        
    if (eval1 == 'bad')  & (pages >5 ) & ((concept_note_cond ==True ) | (project_document_cond ==True )):
        eval2 = 'ok'
        
    if (eval1 == 'bad')  & (pages >2 ) & (concept_note_cond ==True ): #new
        eval2 = 'ok'

    if (eval1 == 'bad') & (pages >5 ) & (pages <11 ) & (awp != True) :
        eval2 = 'review'
        
    if (eval1 == 'bad') & (pages >5 ) & (pages <11 ) & (awp == True) :
        eval1 = 'bad'
    
    if (content_missing == "review") & (eval2 == "not ok") :
        eval2 = "review"

    
    dfcsv.loc[index,'eval1'] = eval1
    dfcsv.loc[index,'eval2'] = eval2
    dfcsv.loc[index,'score'] = score
    dfcsv.loc[index,'pages'] = pages
    dfcsv.loc[index,'project_title_cond'] = project_title_cond
    dfcsv.loc[index,'summary_desc_cond'] = summary_desc_cond
    dfcsv.loc[index,'project_id_cond'] = project_id_cond
    dfcsv.loc[index,'implementing_partner_cond'] = implementing_partner_cond
    dfcsv.loc[index,'start_date_cond'] = start_date_cond
    dfcsv.loc[index,'end_date_cond'] = end_date_cond
    dfcsv.loc[index,'budget_cond'] = budget_cond
    dfcsv.loc[index,'concept_note_cond'] = concept_note_cond
    dfcsv.loc[index,'project_document_cond'] = project_document_cond
    dfcsv.loc[index,'awp_cond'] = awp_cond
    
    #print(eval1,eval2)
    print(page_content[0:100])

    print(str(i) + ": " + eval2)
    

https://undpngddlsprod01.blob.core.windows.net/pdc/Project Document ESPECRP IDA 19 FINAL.doc.pdf
docusign envelope id: dc6b6234-539d-495e-b46d-ea2b 15bc9908

united nations development programme

p
1: ok
https://undpngddlsprod01.blob.core.windows.net/pdc/Initiation Plan GCP Phase II.pdf
29/11/2019

un development programme
regional centre - panama

project: 00123562 budget (us$) as of 
2: ok
https://undpngddlsprod01.blob.core.windows.net/pdc/BRA14G31 - 2021.05.28 - General Revision 13 (simplified).pdf
docusign envelope id: 0b9bd241-1a68-403e-aab3-caa28c86430d

27/5/2021

un development programme

bra
3: ok
https://undpngddlsprod01.blob.core.windows.net/pdc/AGR ILO UNPRPD - Output 00130305.pdf
docusign envelope id: d894e64b-143f-48e8-a5bb-db489582dabb

un to un transfer agreement

a. summary 
4: ok
https://undpngddlsprod01.blob.core.windows.net/pdc/BRA14G31 - Revision 2022.06.30 - General 14 simpl signed.pdf
docusign envelope id: 2f204fb8-8285-49e6-9655-2347d7f2af77
30/6/2022

(en
un de

Multiple definitions in dictionary at byte 0x219acd4 for key /Info
Multiple definitions in dictionary at byte 0x219ace2 for key /Info
Multiple definitions in dictionary at byte 0x219acf0 for key /Info


get

united nations development programme
project document

project title: development of the fifth 
129: ok
https://undpngddlsprod01.blob.core.windows.net/pdc/Project Document - NDS 3 SMF.pdf
project title: support for development of north macedonia national development
strategy (nds) — phas
130: review
https://undpngddlsprod01.blob.core.windows.net/pdc/00136407-prodoc-JP Gender3.pdf
docusign envelope id: 4a65096d-7ebd-4faa-9cdc-67ac27d56283

annex a

1 cover page

country: georgia 
131: ok
https://undpngddlsprod01.blob.core.windows.net/pdc/Agreement for the provision of services FINAL .docx.pdf
docusign envelope id: 81e3e989-3b7a-4f20-9fod-ea62cf6b1de6

agreement for the provision of services 
132: not ok
https://undpngddlsprod01.blob.core.windows.net/pdc/PREPER.PDF
docusign envelope id: 18c76a59-9c68-4127-88de-4090cd9f8ed5

united nations development programme

co
133: ok
https://undpngddlsprod01.blob.core.windows.net/pdc/134183_Revisión C.pdf
docusign envelope id: 2ccofc5b-100e-417

In [ ]:
# Identify duplicate records based on a specific column (e.g., 'ID')
duplicate_records = dfcsv[dfcsv.duplicated(subset='id', keep=False)]

# Add columns representing the two values of another column (e.g., 'Value')
duplicate_records['v1'] = duplicate_records.groupby('id')['eval1'].transform('first')
duplicate_records['v2'] = duplicate_records.groupby('id')['eval1'].transform('last')

# Display the duplicate records with added columns
print(duplicate_records)

In [ ]:
duplicate_records.to_csv('duplicates.csv',index=False)

In [13]:
#changed from 'lambda x: x.zfill(8)' to 'lambda x: str(x).zfill(8)'
dfcsv['project_id'] = dfcsv['project_id'].apply(lambda x: str(x).zfill(8))
dfcsv.to_csv('documents.csv',index=False)

In [16]:


dfcsv = pd.read_csv('documents.csv')

#Filter for date of last run

# dfcsv.query('last_update == "12/02/24"   ')


#count bad last run

# dfcsv.query('last_update == "12/02/24" AND eval1 == "bad"   ')


#Show bad last run
dfcsv_bad = dfcsv.query('last_update == "2024-02-19" and eval1 == "bad"   ')
dfcsv_good = dfcsv.query('last_update == "2024-02-19" and eval1 == "good"   ')


print("Num of bad: ",  len(dfcsv_bad))
print("Num of good: ",  len(dfcsv_good))






Num of bad:  58
Num of good:  268


In [ ]:

#count url ALL FALSE last run

print("Urls Bad ALL FALSE")

dfcsv.query('last_update == "2024-02-19" and eval1 == "bad" and project_title_cond == 0 and summary_desc_cond == 0 and project_id_cond == 0 and implementing_partner_cond == 0 and start_date_cond == 0 and	end_date_cond == 0 and budget_cond == 0 and	concept_note_cond == 0 and  project_document_cond == 0 and awp_cond == 0   ')



In [ ]:
dfcsv_review = dfcsv.query('last_update == "2024-02-19" and eval2 == "review"   ')

print("Num of review: ",  len(dfcsv_review))

dfcsv_review